# Budget closure in MOM6
In this tutorial, we outline the terms required to close the heat, salt and thickness budgets in MOM6. We show this for diagnostics that are output both on the native grid and remapped to a diagnostic grid ($z^*$ and $\rho_2$).

## Conservation of vertically-extensive tracer content
Conservation of tracer concentration $\phi$ can be written
$$ \rho\frac{D\phi}{Dt} = -\nabla\cdot \mathbf{J}_\phi + \rho S_{\phi}$$
where $D/Dt$ is the material derivative, $\mathbf{J}_\phi$ is the diffusive (non-advective) tracer flux, and $S_{\phi}$ are sources/sinks.  

Due to its Lagrangian vertical coordinate, MOM6 is formulated to conserve the vertically-extensive (concentration times thickness) tracer content in each layer at each point on the horizontal grid. Integrated vertically in a model layer, the material derivative in the above expression takes the semi-continuous form:
$$ \int^{z_{k-1}}_{z_k}\Big(\rho\frac{D\phi}{Dt}\Big)\,dz = \partial_t (\rho \phi h) + \nabla_s \cdot (\rho \phi \mathbf{u}\, h) + \Delta_k(\rho \phi w^{(s)})\,,$$
where $\partial_t$ is the Eulerian time derivative, $h$ is the layer thickness, $\nabla_s$ is the gradient operator parallel to the general vertical coordinate $s$, $\mathbf{u}$ is the along-layer velocity, $\Delta_k$ is a discrete difference operator between the upper and lower bounds of the layer, and $w^{(s)}$ is the across-layer velocity. Setting the tracer concentration, $\phi$, to 1, this become an expression for mass conservation (equivalent to thickness conservation under the Boussinesq approximation and with fixed horizontal grid spacing).

In closing the MOM6 budget, the first term on the right hand side of this expression corresponds to the total tendency at the grid cell center, the second term  corresponds to the tendency due to the divergence of along-layer advection, and the third term corresponds to the tendency due to the *vertical remapping*.  

Correspondingly, the vertically-integrated right hand side of the tracer concentration conservation equation can be written
$$\int^{z_{k-1}}_{z_k}(-\nabla\cdot \mathbf{J}_\phi)\,dz = -\nabla_s\cdot(\mathbf{J}_\phi\,h)-\Delta_k(\mathbf{J}^{(s)}_\phi) + \rho S_{\phi}\,h,$$
where $\mathbf{J}^{(s)}_\phi$ is the vertical component of the tracer flux.

#### Non-local isoneutral diffusion and the challenge of closing budgets from fluxes
Isoneutral diffusion in the arbitrary vertical coordinate of MOM6 is handled in such a way that means tracer fluxes can be "non-local". That is, a diffusive flux of tracer for example in the $x$-direction out of layer $k$ at horizontal grid-point $(i,j)$ need not go directly into layer $k$ at horizontal grid-point $(i+1,j)$. Rather, it can go into any layer at grid-point $(i+1,j)$ that satifies the conditions for iso-neutrality.  

This makes it, at present, impossible to fully close layer-wise tracer budgets using the diffusive fluxes at a grid-cell's lateral boundaries, since they are closed only in a column-integrated sense. Instead, we can only fully close the layerwise budgets from the *tendencies* of the tracer in the grid-cell center. That is because, even the keeping track of lateral isoneutral fluxes is impossible, we can still calculate the *total tendency* due to these fluxes at each point and in each layer.  

Note the that colum-integrated tracer budget *can* be closed from the lateral flux and source terms directly.

## Terms in tracer budget
The processes influencing the heat, salt, and thickness budgets in MOM6, and the standard diagnostic names for their corresponding tendencies are given in the following table:  

| Process                            | Heat                             | Salt                             | Thickness                     |
| :-------------                     | :----------                      | :-------                         | :-------                      |
| Eulerian time tendency             | `opottemptend`                   | `osalttend`                      | `dhdt`                        |
| Along-layer advection              | `T_advection_xy`                 | `S_advection_xy`                 | `dynamics_h_tendency`         |
| Across-layer advection (remapping) | `Th_tendency_vert_remap`         | `Sh_tendency_vert_remap`         | `vert_remap_h_tendency`       |
| Boundary forcing                   | `boundary_forcing_heat_tendency` | `boundary_forcing_salt_tendency` | `boundary_forcing_h_tendency` |
| Iso-neutral diffusion              | `opottemppmdiff`                 | `osaltpmdiff`                    | N/A                           |
| Across-layer diffusion             | `opottempdiff`                   | `osaltdiff`                      | N/A                           |
| Internal (geothermal) heating      | `internal_heat_heat_tendency`    | N/A                              | N/A                           |
| Frazil ice formation               | `frazil_heat_tendency`           | N/A                              | `internal_heat_h_tendency`    |


## Showing budget closure
Here, we confirm that, given the terms in the table above, the left hand side (material derivative) and right hand side (diffusive fluxes and source/sink) of the tracer conservation equation are equal and thus sum to zero (to machine precision). We confirm this for (a) the full depth integral, (b) a single profile, and (c) integrated within each layer (full 3D closure).  

For this purpose we look at a simple, Baltic Sea configuration of OM4, available [here](https://github.com/NOAA-GFDL/MOM6-examples/tree/dev/gfdl/ice_ocean_SIS2/Baltic_OM4_025). There is no geothermal heating in this configuration, so the `internal_heat_*` is not part of either the heat or thickness budgets. We consider monthly averages of the tendency terms.

In [1]:
# Import packages
import xarray as xr
from matplotlib import pyplot as plt

In [19]:
# Specify budget terms
terms = {}
terms['heat'] = ['opottemptend','T_advection_xy','Th_tendency_vert_remap',
                 'boundary_forcing_heat_tendency','opottempdiff','opottemppmdiff','frazil_heat_tendency']
terms['salt'] = ['osalttend','S_advection_xy','Sh_tendency_vert_remap',
                 'boundary_forcing_salt_tendency','osaltdiff','osaltpmdiff']
terms['thickness'] = ['dhdt','dynamics_h_tendency','vert_remap_h_tendency',
                      'boundary_forcing_h_tendency','internal_heat_h_tendency']
# Specify different diagnostic grids
grids = ['native','z','rho2']

### Diagnostics output on the native grid
MOM6 has the capability of outputting diagnostics either on the native grid (the grid on which the equations are solved) or a user-defined diagnostic grid, where the diagnostics have been remapped onto, for example, $z^*$ or $\rho_2$ levels.

In [20]:
# Load data
rootdir = '/archive/gam/MOM6-examples/ice_ocean_SIS2/Baltic_OM4_025/1yr/'
filename = '19000101.ocean_daily'
grid = grids[0]
ds = {}
ds['native'] = xr.open_dataset(rootdir+filename+'_'+grid+'.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/archive/gam/MOM6-examples/ice_ocean_SIS2/Baltic_OM4_025/1m_ashao-new/19000101.ocean_daily_native.nc'

In [18]:
# Define a function to calculate the two sides of the tracer conservation equation
def calc_budget(ds,terms):
    lhs = xr.Dataset()
    rhs = xr.Dataset()
    for key in terms.keys():
    #     lhs[key] = xr.zeros_like(ds[terms[key][0]])
    #     rhs[key] = xr.zeros_like(ds[terms[key][3]])
        for term in terms[key][0:3]:
            # Flip the sign for the Eulerian time tendency
            if term in ['opottemptend','osalttend','dhdt']:
                sign=-1
            else:
                sign=1
    #         lhs[key] += sign*ds[term]
        for term in terms[key][3:]:
            sign=1
    #         rhs[key] += sign*ds[term]
    return lhs, rhs

heat
 opottemptend
  yes
 T_advection_xy
 Th_tendency_vert_remap
salt
 osalttend
  yes
 S_advection_xy
 Sh_tendency_vert_remap
thickness
 dhdt
  yes
 dynamics_h_tendency
 vert_remap_h_tendency


In [ ]:
# Calculate the two sides of the tracer equation on the native grid
lhs = {}
rhs = {}
lhs['native'],rhs['native'] = calc_budget(ds['native'],terms)

In [ ]:
# Plot the vertical sum
fig,ax = plt.subplots(figsize=(16,16))
count=0
for key in terms.keys():
    # LHS
    im = ax[count,0].pcolormesh(lhs[key].sum('zl'))
    plt.colorbar(im,ax=ax[0,count])
    ax[count,0].set_title(key)
    # RHS
    ax[count,1].pcolormesh(rhs[key].sum('zl'))
    plt.colorbar(im,ax=ax[1,count])
    # Difference
    ax[count,2].pcolormesh((lhs[key]-rhs[key]).sum('zl'))
    plt.colorbar(im,ax=ax[2,count])
plt.tight_layout()

In [ ]:
# Plot a profile
# [In this case, plot the terms against the vertical sum of the cell thicknesses, so that the y-axis is depth]
fig,ax = plt.subplots(figsize=)

## Diagnostics output on a user-defined vertical grid
In MOM6, diagnostics can be output on a user-defined vertical grid, such as $z^*$ levels or $\rho_2$ surfaces. This is done through a remapping procedure and it is important to confirm that the procedure does not break the budget closure.  

In [ ]:
# Load data for z* and rho2 output
for grid in grids[1:]:
    ds[grid] = xr.open_dataset(rootdir+filename+'_'+grid+'.nc')
    # Calculate the two sides of the tracer equation on the native grid
    lhs[grid],rhs[grid] = calc_budget(ds[grid],terms)